In [1]:
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import numpy as np

In [2]:
path_to_data = 'C:/Users/PetukhovMD/Desktop/szp/'
path_to_docs = 'C:/Users/PetukhovMD/Desktop/справочники/'
months = ['feb.xlsx', 'jan.xlsx']
sheet = 'Sheet'

In [3]:
def to_double(row, f_name):
    if type(row[f_name]) is float:
        return row[f_name] 
    t = str(row[f_name]).split(',')
    if t[0] == 'nan':
        return 0.0
    return float(t[0]+'.'+t[1])

In [4]:
def print_df(df, name):
    wb = Workbook()
    ws = wb.active
    for  i in df_to_row(df, header=True):
        ws.append(i)
    wb.save(name + '.xlsx')

In [16]:
def full_unclean(months):
    for month in months:
        df = pd.read_excel(path_to_data + month + '.xlsx')
        df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
        df['Причина'] = df.apply(lambda row: unclean(row), axis=1)
        print_df(df, 'unclean_' + month)

In [17]:
full_unclean(['jan', 'feb'])

In [12]:
df = pd.read_excel(path_to_data + 'feb' + '.xlsx')
df.stv = df.apply(lambda row: to_double(row, 'stv'), axis=1)
df['Причина'] = df.apply(lambda row: unclean(row), axis=1)
df['date'] = 'feb'
df

,inn,snils,sum,job,day,type,stv,status,Причина,date
0,7734408766,001-879-913 57,119742.67,Преподаватель,1,Основное место работы,1.0,Работа,Норм запись,feb
1,7734408766,001-896-420 39,142964.10,Преподаватель,1,Основное место работы,1.0,Работа,Норм запись,feb
2,7734408766,002-050-383 67,68007.79,Преподаватель,1,Основное место работы,1.0,Работа,Норм запись,feb
3,7734408766,002-065-271 85,91097.30,Преподаватель,1,Основное место работы,1.0,Работа,Норм запись,feb
4,7734408766,002-845-525 25,114596.10,Преподаватель,1,Основное место работы,1.0,Работа,Норм запись,feb
...,...,...,...,...,...,...,...,...,...,...
199468,7722069597,186-373-917 14,67494.74,Специалист по интернет-маркетингу,0,Основное место работы,1.0,Работа,Не полный месяц,feb
199469,7722069597,187-703-552 01,62968.42,Воспитатель,0,Основное место работы,1.0,Болезнь,Не полный месяц,feb
199470,7722069597,198-391-256 24,91973.89,Воспитатель,1,Основное место работы,1.0,Работа,Норм запись,feb
199471,7722069597,202-693-820 52,73130.37,Учитель,0,Основное место работы,1.0,Болезнь,Не полный месяц,feb


In [11]:
def unclean(row):
    if row['type'] == 'Внутреннее совместительство':
        return 'Внутреннее совместительсьво, сливается в основное'
    if row['type'] not in ['Внутреннее совместительство', 'Основное место работы']:
        return 'Тип работы не основа и не внутр.совм.'
    if row['job'] == 'no job value':
        return 'Увольнение/прием в середине месяца'
    if row['day'] == 0:
        return 'Не полный месяц'
    if row['stv'] < 1:
        return 'Меньше ставки'
    return 'Норм запись'

In [15]:
df[(df.day == 1) & (df.status != 'Работа') & (df.job != 'no job value') & (df.Причина == 'Норм запись')]

,inn,snils,sum,job,day,type,stv,status,Причина,date
465,7720016173,128-323-994 69,134689.22,Учитель,1,Основное место работы,1.00,Болезнь,Норм запись,feb
1165,7736076587,125-137-982 53,63000.00,Воспитатель,1,Основное место работы,1.00,Работа в отпуске по уходу за ребенком,Норм запись,feb
1171,7736076587,125-921-967 73,32842.59,Воспитатель,1,Основное место работы,1.00,Отпуск по уходу за ребенком,Норм запись,feb
1707,7717043346,001-592-664 23,68703.89,Библиотекарь,1,Основное место работы,1.00,Болезнь,Норм запись,feb
1997,7717043346,012-726-200 03,57469.74,Библиотекарь,1,Основное место работы,1.00,Болезнь,Норм запись,feb
...,...,...,...,...,...,...,...,...,...,...
195676,7726317748,128-792-726 05,70125.00,Учитель-логопед,1,Основное место работы,1.00,Работа в отпуске по уходу за ребенком,Норм запись,feb
195719,7726317748,134-201-474 06,125739.13,Специалист,1,Основное место работы,1.00,Отпуск основной,Норм запись,feb
196107,7713229928,060-418-662 40,160771.68,Учитель,1,Основное место работы,1.00,Болезнь,Норм запись,feb
196248,7713229928,138-083-959 86,113667.56,Учитель,1,Основное место работы,1.00,Болезнь,Норм запись,feb
